In [ ]:
import re
import pickle
import random

from collections import defaultdict

import torch
import numpy as np

import random

In [ ]:
trans = str.maketrans('(),_-/', '      ','\\"')

#### GET VOCAB AND PAGES

In [ ]:
pages_dict = dict()

word_cnt = 1 # 0 reserved
word2idx = dict()
idx2word = dict()

In [ ]:
with open(r'C:\Users\Tadija\Desktop\wikipedia\page.sql', 'r', encoding='utf8') as _if:
    
    for i, line in enumerate(_if):
        
        if (i + 1) % 100 == 0:
            print('processed line {}'.format(i+1))
        
        
        matches = re.findall('\(([^,]*),0,\'(.*?)\',.*?\)', line)
    
        for match in matches:    
            title = match[1].translate(trans).split()
            
            if len(title) > 10:
                continue
            
            for token in title:
                if token not in word2idx:
                    word2idx[token] = word_cnt
                    idx2word[word_cnt] = token
                    
                    word_cnt += 1
            
            title = [word2idx[token] for token in title]
            
            pages_dict[int(match[0])] = title

#### SAVE

In [ ]:
with open('C:/Users/Tadija/Desktop/wikipedia/pickle/word2idx.pickle', 'wb') as of:
    pickle.dump(word2idx, of)
    
with open('C:/Users/Tadija/Desktop/wikipedia/pickle/idx2word.pickle', 'wb') as of:
    pickle.dump(idx2word, of)
    
with open('C:/Users/Tadija/Desktop/wikipedia/pickle/pages_dict.pickle', 'wb') as of:
    pickle.dump(pages_dict, of)    
    

#### LOAD

In [ ]:
with open('C:/Users/Tadija/Desktop/wikipedia/pickle/word2idx.pickle', 'rb') as _if:
    word2idx = pickle.load(_if)
    
with open('C:/Users/Tadija/Desktop/wikipedia/pickle/idx2word.pickle', 'rb') as _if:
    idx2word = pickle.load(_if)
    
with open('C:/Users/Tadija/Desktop/wikipedia/pickle/pages_dict.pickle', 'rb') as _if:
    pages_dict = pickle.load(_if)
    
word_cnt = len(word2idx)

#### GET LINKS

In [ ]:
links_dict = defaultdict(list)

In [ ]:
with open(r'C:\Users\Tadija\Desktop\wikipedia\pagelinks.sql','r', encoding='utf8', errors='ignore') as _if:
    for i, line in enumerate(_if):
        
        if i < 18000:
            continue
            
        if i == 24490:
            break
            
        if (i + 1) % 1000 == 0:
            print('processed line {}'.format(i+1))
        
        matches = re.findall('\(([0-9]*),0,\'(.*?)\',0\)', line)

        for match in matches:
            if int(match[0]) in pages_dict:
                
                title = match[1].translate(trans).split()
                
                if len(title) > 10:
                    continue
                    
                to_continue = False
                
                for token in title:
                    if token not in word2idx:
                        to_continue = True
                        break
                
                if to_continue:
                    continue      
                        
                title = [word2idx[token] for token in title]
            
                links_dict[tuple(pages_dict[int(match[0])])].append(title)

In [ ]:
with open('C:/Users/Tadija/Desktop/wikipedia/pickle/links_dict4.pickle', 'rb') as _if:
    links_dict = pickle.load(_if)

In [ ]:
xs = []
ys = []

cutoff = 8388608
acc = 0
ind = 55

for key, value in links_dict.items():
    
    if len(key) > 8:
        continue
    
    
    cnt = 0
    for v in value:
        if len(v) > 0 and len(v) <= 8:
            cnt += 1
    
    x = np.zeros((cnt, 8), dtype=np.int64)
    y = np.zeros((cnt, 8), dtype=np.int64)
    
    x[:,:len(key)] = key
    
    idx = 0 
    for i in range(len(value)):
        if len(value[i]) > 0 and len(value[i]) <= 8:
            y[idx,:len(value[i])] = value[i]
            idx += 1

    xs.append(x)
    ys.append(y)
    
    acc += cnt
                   
    if acc > cutoff:
        xs = np.vstack(xs)
        ys = np.vstack(ys)
    
        np.save('C:/Users/Tadija/Desktop/wikipedia/numpy/xs{}.np'.format(ind), xs)
        np.save('C:/Users/Tadija/Desktop/wikipedia/numpy/ys{}.np'.format(ind), ys)

        acc = 0
        ind += 1
        
        del xs
        del ys
        
        xs = []
        ys = []

In [ ]:
xs = np.vstack(xs)
ys = np.vstack(ys)

np.save('C:/Users/Tadija/Desktop/wikipedia/numpy/xs{}.np'.format(ind), xs)
np.save('C:/Users/Tadija/Desktop/wikipedia/numpy/ys{}.np'.format(ind), ys)

acc = 0
ind += 1

del xs
del ys

xs = []
ys = []

In [ ]:
indices = [i for i in range(0, 68)]

random.shuffle(indices)

indices_groups = []

for i in range(7):
    indices_groups.append(indices[ i*10 : ((i+1)*10 if (i+1)*10 < 68 else 68) ])

In [ ]:
indices_groups = [[25, 62, 20, 7, 51, 64, 6, 3, 59, 19],
 [54, 44, 0, 46, 37, 50, 5, 66, 11, 18],
 [32, 55, 24, 43, 27, 48, 12, 17, 16, 21],
 [60, 65, 47, 34, 39, 13, 9, 31, 14, 36],
 [45, 63, 41, 58, 29, 26, 57, 49, 1, 30],
 [53, 40, 2, 42, 33, 4, 67, 28, 35, 22],
 [38, 52, 15, 61, 10, 56, 8, 23]]

In [ ]:
for i, group in enumerate(indices_groups):
    xs = []
    ys = []
    
    for ind in group:
        
        xs.append(np.load('C:/Users/Tadija/Desktop/wikipedia/numpy/xs{}.np.npy'.format(ind)))
        ys.append(np.load('C:/Users/Tadija/Desktop/wikipedia/numpy/ys{}.np.npy'.format(ind)))
        
    xs = np.vstack(xs)
    ys = np.vstack(ys)
    
    dataset = torch.utils.data.TensorDataset(torch.LongTensor(xs), torch.LongTensor(ys))
    
    torch.save(dataset, 'C:/Users/Tadija/Desktop/wikipedia/tensor/dataset{}.pt'.format(i))
    
    del xs
    del ys
    del dataset

In [ ]:
def load_embeddings(fname, get_embeddings=True, get_w2i=False, get_i2w=False, skip_first_line=True):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    
    if skip_first_line:
        fin.readline()
    
    num_embeddings = 0
    
    word2idx = {}
    idx2word = {}

    embeddings = []

    for line in fin:
        line = line.rstrip().split(' ')
        
        if get_w2i:
            word2idx[line[0]] = num_embeddings
        if get_i2w:
            idx2word[num_embeddings] = line[0]
        if get_embeddings:       
            embeddings.append([float(num) for num in line[1:]])
        
        num_embeddings += 1
        
        
    return torch.FloatTensor(embeddings), word2idx, idx2word

In [ ]:
embeddings_used

In [ ]:
embeddings_used = 0

In [ ]:
embeddings = torch.FloatTensor(len(word2idx)+1, 300)

torch.nn.init.xavier_normal_(embeddings)

embeddings[0] = torch.zeros(300)

In [ ]:
import io

fin = io.open(r'C:\Users\Tadija\Desktop\word embeddings\glove.840B.300d.txt', 'r', encoding='utf-8', newline='\n', errors='ignore')

for line in fin:
    line = line.rstrip().split(' ')
    
    if line[0] in word2idx:
        embeddings_used += 1
        
        embeddings[word2idx[line[0]]] = torch.FloatTensor([float(num) for num in line[1:]])

fin.close()

In [ ]:
torch.save(embeddings, r'C:\Users\Tadija\Desktop\wikipedia\tensor\embeddings_init.pt')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(len(a)), a, 'rx')